## 보스턴 집값 예측

<img src="assets/boston.png">

- 1970년대 매사추세츠주 보스턴에서 우리가 부동산 개발 회사에서 일하고 있다고 기장헤보자. 주거지 프로젝트를 시작하기 전에 평가해 보려고 한다. 우리는 아래와 같이 집의 특성에 따라 가격 견적서를 제공하는 모델을 만들어야 한다.
    - 방의 개수
    - 고용 센터까지의 거리
    - 지역의 경제력 수준
    - 지역 학교의 교사당 학생 수 등

- 우리가 이번 시간을 통해 배우거나 작업할 사항은 아래와 같다. 
    1. 보스턴 집의 가격 데이터를 분석, 연구
    2. 데이터를 훈련용과 테스트용으로 나누기
    3. 다변수 회귀 분석을 수행
    4. 모델의 계수와 잔차 평가
    5. 모델 성능을 개선하기 위해 데이터 변환 사용
    6. 모델을 이용해 부동산 가격을 감정
    
<img src="assets/model.png">